# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
# import libraries
from sqlalchemy import create_engine 
import nltk
import pandas as pd
import sqlite3
import pickle

import re
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [4]:
# load data from database
from sqlalchemy import create_engine, text
engine = create_engine('sqlite:///Disaster_Analysis.db')
query = 'SELECT * FROM Messages_with_Cats'
df = pd.read_sql_query(sql=text(query), con=engine.connect())
# Choose the X and Y (feature and target variables) for ML Model
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    
    '''
    INPUT 
        text: Text to be processed   
    OUTPUT
        Returns an array of words after being processed; by being tokenized, lower cased, stripped, and lemmatized
    '''
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    
    '''
    Define a custom estimator that extract the feature from a sentence checking if the text is starting with a verb
    
    '''

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [7]:
def model_pipeline():
    
    pipeline = Pipeline([
            ('features', FeatureUnion([

                ('text_pipeline', Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer())
                ]))
            ])),
        
            ('clf', MultiOutputClassifier(KNeighborsClassifier()))
        ])
    return pipeline

In [8]:
def model_pipeline_improved():
    
    pipeline = Pipeline([
            ('features', FeatureUnion([

                ('text_pipeline', Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer())
                ])),

                ('starting_verb', StartingVerbExtractor())
            ])),

            ('clf', MultiOutputClassifier(KNeighborsClassifier(n_neighbors=10)))
        ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# Define the model and fit the training dataset
model = model_pipeline()
model.fit(X_train, y_train)

c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x00000222DDC7D900>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())]))])),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
# Predict from trained model using the test dataset
y_pred = model.predict(X_test)

In [11]:
print(classification_report(y_test.values, y_pred,target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.95      0.87      4984
               request       0.69      0.49      0.57      1105
                 offer       1.00      0.04      0.08        25
           aid_related       0.66      0.53      0.59      2694
          medical_help       0.28      0.18      0.22       507
      medical_products       0.23      0.18      0.20       325
     search_and_rescue       0.62      0.06      0.11       166
              security       0.25      0.01      0.02       127
              military       0.52      0.11      0.18       218
           child_alone       0.00      0.00      0.00         0
                 water       0.47      0.33      0.39       443
                  food       0.46      0.42      0.43       719
               shelter       0.37      0.29      0.32       599
              clothing       0.75      0.13      0.23       112
                 money       0.37      

c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {
    'clf__estimator__n_neighbors': [3, 7, 10],
    'clf__estimator__weights': ['uniform', 'distance']
}

# Create new model with same pipeline and apply grid search on it
model2 = model_pipeline()

cv = GridSearchCV(model2, param_grid = parameters, cv=2, scoring='accuracy', verbose=10)

In [13]:
cv.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__features', 'estimator__clf', 'estimator__features__n_jobs', 'estimator__features__transformer_list', 'estimator__features__transformer_weights', 'estimator__features__verbose', 'estimator__features__text_pipeline', 'estimator__features__text_pipeline__memory', 'estimator__features__text_pipeline__steps', 'estimator__features__text_pipeline__verbose', 'estimator__features__text_pipeline__vect', 'estimator__features__text_pipeline__tfidf', 'estimator__features__text_pipeline__vect__analyzer', 'estimator__features__text_pipeline__vect__binary', 'estimator__features__text_pipeline__vect__decode_error', 'estimator__features__text_pipeline__vect__dtype', 'estimator__features__text_pipeline__vect__encoding', 'estimator__features__text_pipeline__vect__input', 'estimator__features__text_pipeline__vect__lowercase', 'estimator__features__text_pipeline__vect__max_df', 'estimator__features__te

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
cv.fit(X_train, y_train)
#cv.get_params()

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV 1/2; 1/6] START clf__estimator__n_neighbors=3, clf__estimator__weights=uniform


c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 1/2; 1/6] END clf__estimator__n_neighbors=3, clf__estimator__weights=uniform;, score=0.210 total time=13.3min
[CV 2/2; 1/6] START clf__estimator__n_neighbors=3, clf__estimator__weights=uniform


c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/2; 1/6] END clf__estimator__n_neighbors=3, clf__estimator__weights=uniform;, score=0.204 total time=11.1min
[CV 1/2; 2/6] START clf__estimator__n_neighbors=3, clf__estimator__weights=distance


...

[CV 1/2; 6/6] END clf__estimator__n_neighbors=10, clf__estimator__weights=distance;, score=0.196 total time=10.2min
[CV 2/2; 6/6] START clf__estimator__n_neighbors=10, clf__estimator__weights=distance


c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/2; 6/6] END clf__estimator__n_neighbors=10, clf__estimator__weights=distance;, score=0.205 total time=19.4min


c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x00000222DDC7D900>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())]))])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=KNeighborsClassifier()))]),
             param_grid={'clf__estimator__n_neighbors': [3, 7, 10],
                         'clf__estimator__weights': ['uniform', 'distance']},
             scoring='accuracy', verbose=10)

In [15]:
y_pred2 = cv.predict(X_test)

In [16]:
best_clf = cv.best_estimator_
best_clf

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x00000222DDC7D900>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())]))])),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier(n_neighbors=10)))])

In [17]:
print(classification_report(y_test.values, y_pred2,target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      4984
               request       0.75      0.50      0.60      1105
                 offer       0.00      0.00      0.00        25
           aid_related       0.68      0.57      0.62      2694
          medical_help       0.30      0.18      0.22       507
      medical_products       0.22      0.16      0.19       325
     search_and_rescue       1.00      0.02      0.04       166
              security       0.00      0.00      0.00       127
              military       0.71      0.05      0.09       218
           child_alone       0.00      0.00      0.00         0
                 water       0.51      0.31      0.39       443
                  food       0.47      0.42      0.44       719
               shelter       0.38      0.26      0.31       599
              clothing       0.75      0.08      0.15       112
                 money       0.64      

c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:
model_improved = model_pipeline_improved()
model_improved.fit(X_train, y_train)

c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x00000222DDC7D900>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('starting_verb',
                                                 StartingVerbExtractor())])),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier(n_neighbors=10)))])

In [19]:
y_pred3 = model_improved.predict(X_test)

In [20]:
print(classification_report(y_test.values, y_pred3,target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.94      0.88      4984
               request       0.75      0.50      0.60      1105
                 offer       0.00      0.00      0.00        25
           aid_related       0.69      0.57      0.62      2694
          medical_help       0.32      0.14      0.20       507
      medical_products       0.28      0.14      0.18       325
     search_and_rescue       0.80      0.02      0.05       166
              security       0.00      0.00      0.00       127
              military       0.67      0.05      0.09       218
           child_alone       0.00      0.00      0.00         0
                 water       0.59      0.28      0.38       443
                  food       0.51      0.40      0.45       719
               shelter       0.45      0.24      0.31       599
              clothing       0.67      0.05      0.10       112
                 money       0.57      

c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

### 9. Export your model as a pickle file

In [21]:
model_filepath = 'classifier.pk'
file = open(model_filepath, 'wb')
pickle.dump(model, file)
file.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.